In [1]:
import nltk
import pandas as pd
import re
 
from sklearn.feature_extraction.text import TfidfVectorizer
import string
import nltk
import pandas as pd
import re
 
from sklearn.feature_extraction.text import TfidfVectorizer
import string
 

data = pd.read_csv("../Data/SMSSpamCollection.txt", sep='\t', header=None)
data.columns = ['label', 'Content']

en_stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data['Content_len'] = data['Content'].apply(lambda x: len(x) - x.count(" "))

def count_punctuation(text):
    binary_array = [1 for ch in text if ch in string.punctuation] 
    nb_ponctuation = sum(binary_array)
    total = len(text) - text.count(" ")
    return round(nb_ponctuation/(total), 4)*100

data['punctuation_rate'] = data['Content'].apply(lambda x: count_punctuation(x))


def clean_email(email):
    result = "".join([word for word in email if word not in string.punctuation])
    tokens = re.split(r'\W+', result)
    text = [ps.stem(word) for word in tokens if word not in en_stopwords]
    return text

 

vectorisation_full = TfidfVectorizer(analyzer=clean_email)
vect_final = vectorisation_full.fit_transform(data['Content'])


all_data = pd.concat([pd.DataFrame(vect_final.toarray()), data['Content_len'], data['punctuation_rate']], axis=1)
all_data

,0,1,2,3,4,5,6,7,8,9,...,8185,8186,8187,8188,8189,8190,8191,8192,Content_len,punctuation_rate
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,92,9.78
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,24,25.00
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,128,4.69
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,39,15.38
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,49,4.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5567,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,131,6.11
5568,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.311086,0.0,0.0,29,3.45
5569,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,48,14.58
5570,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,100,1.00


In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split

In [3]:
all_data.columns = all_data.columns.astype(str)

X_train, X_test, y_train, y_test = train_test_split(all_data, data['label'], test_size=0.2)

In [4]:
def random_forest_hyper_params(n_decision_tree, depth):
    alg_RandomForest = RandomForestClassifier(
        n_estimators=n_decision_tree, max_depth=depth, n_jobs=-1
    )
    model = alg_RandomForest.fit(X_train, y_train)
    predictions = model.predict(X_test)
    precision, recall, fscore, support = score(
        y_test, predictions, pos_label="spam", average="binary"
    )
    print(
        "nb decision tree: {} / Depth: {} | Precision: {} / Recall: {}".format(
            n_decision_tree, depth, round(precision, 3), round(recall, 3)
        )
    )

In [7]:
for ith_decision_tree in [100, 150, 200]:
    for depth in [30,40,50]:
        random_forest_hyper_params(ith_decision_tree, depth)

nb decision tree: 100 / Depth: 30 | Precision: 1.0 / Recall: 0.718
nb decision tree: 100 / Depth: 40 | Precision: 1.0 / Recall: 0.748
nb decision tree: 100 / Depth: 50 | Precision: 1.0 / Recall: 0.771
nb decision tree: 150 / Depth: 30 | Precision: 1.0 / Recall: 0.725
nb decision tree: 150 / Depth: 40 | Precision: 1.0 / Recall: 0.748
nb decision tree: 150 / Depth: 50 | Precision: 1.0 / Recall: 0.756
nb decision tree: 200 / Depth: 30 | Precision: 1.0 / Recall: 0.733
nb decision tree: 200 / Depth: 40 | Precision: 1.0 / Recall: 0.748
nb decision tree: 200 / Depth: 50 | Precision: 1.0 / Recall: 0.763


In [6]:
for ith_decision_tree in [100,200]:
    for depth in [50,100]:
        random_forest_hyper_params(ith_decision_tree, depth)

nb decision tree: 100 / Depth: 50 | Precision: 1.0 / Recall: 0.763
nb decision tree: 100 / Depth: 100 | Precision: 1.0 / Recall: 0.832
nb decision tree: 200 / Depth: 50 | Precision: 1.0 / Recall: 0.756
nb decision tree: 200 / Depth: 100 | Precision: 1.0 / Recall: 0.794


In [8]:
for ith_decision_tree in [200,300]:
    for depth in [100,200]:
        random_forest_hyper_params(ith_decision_tree, depth)

nb decision tree: 200 / Depth: 100 | Precision: 1.0 / Recall: 0.809
nb decision tree: 200 / Depth: 200 | Precision: 1.0 / Recall: 0.809
nb decision tree: 300 / Depth: 100 | Precision: 1.0 / Recall: 0.817
nb decision tree: 300 / Depth: 200 | Precision: 1.0 / Recall: 0.809


In [9]:
for ith_decision_tree in [300,400]:
    for depth in [200,300]:
        random_forest_hyper_params(ith_decision_tree, depth)

nb decision tree: 300 / Depth: 200 | Precision: 1.0 / Recall: 0.802
nb decision tree: 300 / Depth: 300 | Precision: 1.0 / Recall: 0.802
nb decision tree: 400 / Depth: 200 | Precision: 1.0 / Recall: 0.817
nb decision tree: 400 / Depth: 300 | Precision: 1.0 / Recall: 0.809
